<a href="https://colab.research.google.com/github/GoeSoft/My_Python_Learn/blob/main/%D0%9D%D0%B5%D0%B9%D1%80%D0%BE_%D1%8D%D0%BA%D0%B7%D0%B0%D0%BC%D0%B5%D0%BD%D0%B0%D1%82%D0%BE%D1%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai beautifulsoup4 lxml

import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

from openai import OpenAI
client = OpenAI()

In [6]:
import requests
from bs4 import BeautifulSoup

doc_url = "https://docs.google.com/document/d/1ziADtbpwWRt-HxPCBixwsZJn_xdw83qnf0v9K8WXeME"

export_url = doc_url + "/export?format=html"
response = requests.get(export_url)
soup = BeautifulSoup(response.content, 'html.parser')
document_text = soup.get_text(separator="\n", strip=True)

print("📄 Текст документа успешно загружен:")
print(document_text[:500] + "...\n")

📄 Текст документа успешно загружен:
История возникновения искусственного интеллекта и великие ученые
Искусственный интеллект
, как и любая другая область человеческого бытия уже обросла мифами. Сейчас нет внятного понимания, с какой точки вести начало истории возникновения ИИ.
Искусственный интеллект как научное направление представляет собой наглядный пример интеграции различных научных областей. Специалисты в естественно-научных областях и вычислительных науках изучают свойства и функционирование живых систем, пользуясь сходными...



In [7]:
def generate_questions(text, num_questions=5):
    prompt = f"""
На основе следующего текста сформулируйте {num_questions} открытых вопросов для проверки понимания содержания.
Вопросы должны быть:
- Чёткими и конкретными
- Направленными на ключевые идеи текста
- Подходящими для устного или письменного ответа
- Сформулированными в нейтральной, профессиональной манере

Текст:
{text}

Выведите только вопросы, каждый с новой строки, без нумерации и дополнительных комментариев.
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.5,
        max_tokens=300
    )
    questions = response.choices[0].message.content.strip().split('\n')
    return [q.strip() for q in questions if q.strip()]

# Генерация вопросов
questions = generate_questions(document_text, num_questions=3)
print("❓ Сгенерированные вопросы:")
for i, q in enumerate(questions, 1):
    print(f"{i}. {q}")

❓ Сгенерированные вопросы:
1. Как искусственный интеллект интегрирует различные научные области и какие методы используются для его изучения?
2. Какие ключевые достижения в области искусственного интеллекта были сделаны в XX веке, и кто из ученых внес наибольший вклад в их развитие?
3. Каково современное понимание искусственного интеллекта и какие задачи он призван решать?


In [8]:
def evaluate_answer(question, reference_text, student_answer):
    system_prompt = """
Вы — эксперт-экзаменатор. Оценивайте ответ кандидата на открытый вопрос по шкале от 0 до 100.

Критерии:
• 90–100: Полный, точный, структурированный ответ. Раскрыты все ключевые аспекты. Нет ошибок.
• 80–89: Ответ в целом верный, но есть небольшие упущения или неточности.
• 70–79: Основная идея верна, но ответ неполный или содержит второстепенные ошибки.
• 60–69: Частично верный ответ, но много пробелов или существенных упущений.
• 50–59: Есть попытка ответить, но мало релевантной информации.
• 0–49: Ответ неверный, не по теме или отсутствует.

Инструкции:
1. Сравните ответ с информацией из исходного текста.
2. Оцените по шкале 0–100.
3. Кратко обоснуйте оценку: укажите сильные стороны и недостатки.
4. В конце укажите: «Оценка: X/100».

НИКОГДА не выходите за пределы 0–100.
"""

    user_prompt = f"""
Исходный текст (для справки):
{reference_text}

Вопрос: {question}

Ответ кандидата: {student_answer}
"""

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        temperature=0.0,
        max_tokens=300
    )
    return response.choices[0].message.content.strip()

In [9]:
# Пример ответа студента (можно заменить на любой)
sample_answer = "Искусственный интеллект — это когда компьютеры могут думать как люди и решать сложные задачи."

print("\n" + "="*60)
print("🧪 Тестовая оценка ответа на первый вопрос")
print("="*60)
print(f"Вопрос: {questions[0]}")
print(f"Ответ: {sample_answer}\n")

evaluation = evaluate_answer(questions[0], document_text, sample_answer)
print(evaluation)


🧪 Тестовая оценка ответа на первый вопрос
Вопрос: Как искусственный интеллект интегрирует различные научные области и какие методы используются для его изучения?
Ответ: Искусственный интеллект — это когда компьютеры могут думать как люди и решать сложные задачи.

Ответ кандидата очень краткий и не раскрывает тему должным образом. Он упоминает, что искусственный интеллект связан с возможностью компьютеров думать как люди и решать сложные задачи, но не объясняет, как именно происходит интеграция различных научных областей и какие методы используются для изучения ИИ. 

Сильные стороны:
- Кандидат правильно указывает на способность ИИ решать сложные задачи, что является важным аспектом.

Недостатки:
- Ответ не структурирован и не содержит ключевых аспектов, таких как интеграция с естественными науками, вычислительными науками, математикой и логикой.
- Не упоминаются конкретные методы, используемые для изучения ИИ, такие как машинное обучение, нейронные сети и другие.
- Ответ не демонстрир